In [1]:
import numpy as np
import pandas as pd
import copy 
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
%matplotlib inline
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import recall_score
import xgboost as xgb
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from operator import itemgetter, attrgetter

Using TensorFlow backend.


In [8]:
# load data and clean

In [20]:
data = pd.read_csv('newloanData.csv')
data.dropna()

,label,loan_amnt,funded_amnt,term_num,int_rate,grade,sub_grade,emp_length,employlen,home_status,...,application_type,acc_now_delinq,nowdelinq,chargeoff_within_12_mths,chargedoffrecord,delinq_amnt,nev_delinq_itv,annual_inc,debt_ratio,loan_amnt_itv
0,0,30000,30000,36,22.35,D,D5,5 years,5,MORTGAGE,...,Joint App,0,0,0,0,0,"(-1,95]",100000.0,0.100000,"(2.6e+04,3.1e+04]"
1,0,40000,40000,60,16.14,C,C4,< 1 year,1,MORTGAGE,...,Joint App,0,0,0,0,0,"(95,100]",45000.0,0.177778,"(3.6e+04,4e+04]"
2,0,20000,20000,36,7.56,A,A3,10+ years,10,MORTGAGE,...,Joint App,0,0,0,0,0,"(-1,95]",100000.0,0.066667,"(1.6e+04,2.1e+04]"
3,0,4500,4500,36,11.31,B,B3,10+ years,10,RENT,...,Individual,0,0,0,0,0,"(-1,95]",38500.0,0.038961,"(999,6e+03]"
4,0,8425,8425,36,27.27,E,E5,3 years,3,MORTGAGE,...,Joint App,0,0,0,0,0,"(95,100]",450000.0,0.006241,"(6e+03,1.1e+04]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1194183,1,30000,30000,60,17.09,D,D1,2 years,2,RENT,...,Individual,0,0,0,0,0,"(-1,95]",80000.0,0.075000,"(2.6e+04,3.1e+04]"
1194184,1,18000,18000,60,30.65,F,F4,10+ years,10,MORTGAGE,...,Individual,0,0,0,0,0,"(-1,95]",77000.0,0.046753,"(1.6e+04,2.1e+04]"
1194185,0,17950,17950,60,30.17,F,F3,10+ years,10,MORTGAGE,...,Individual,0,0,0,0,0,"(95,100]",53758.0,0.066781,"(1.6e+04,2.1e+04]"
1194186,0,15000,15000,60,16.02,C,C5,< 1 year,1,RENT,...,Individual,0,0,0,0,0,"(-1,95]",85000.0,0.035294,"(1.1e+04,1.6e+04]"


In [67]:
# feature engineering: created a new feature to measure the pressure of paying back the loan.
data['debt_coveredratio']=data['annual_inc']/12/(data['loan_amnt']/data['term_num'])

In [68]:
onehot=['verification_status','purpose', 'application_type','addr_state']
ordinal=['sub_grade']
tostandard=['pct_tl_nvr_dlq', 'loan_amnt', 'funded_amnt', 'term_num', 'int_rate', 'employlen','delinq_2yrs', 'total_acc','chargeoff_within_12_mths', 'delinq_amnt', 'annual_inc','debt_coveredratio']

In [69]:
keep=onehot+ordinal+tostandard+['label']

In [112]:
# transform data: using one-hot encoding, ordinal encoding and standardization.
standardizer=preprocessing.StandardScaler()
onehot_encoder=preprocessing.OneHotEncoder(sparse=False)
ordinal_encoder=preprocessing.OrdinalEncoder()
subdata=data.loc[:,keep]
data1=copy.copy(subdata)
data1[tostandard]=standardizer.fit_transform(data1[tostandard])
onehotdata=onehot_encoder.fit_transform(data1.loc[:,onehot])
onehotdata

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [111]:
with_onehot = pd.DataFrame(onehotdata)
with_onehot.columns=onehot_encoder.get_feature_names()
with_onehot

,x0_Not Verified,x0_Source Verified,x0_Verified,x1_car,x1_credit_card,x1_debt_consolidation,x1_educational,x1_home_improvement,x1_house,x1_major_purchase,...,x3_SD,x3_TN,x3_TX,x3_UT,x3_VA,x3_VT,x3_WA,x3_WI,x3_WV,x3_WY
0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1194183,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1194184,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1194185,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1194186,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
data1[ordinal]=ordinal_encoder.fit_transform(data1[ordinal])
without_onehot=data1.drop(columns=onehot)
combind_data= pd.concat([without_onehot,with_onehot],axis=1)
combind_data.to_csv("onehotdata.csv",index=False)
combind_data=pd.read_csv("onehotdata.csv",index_col=0)

In [10]:
combind_data

,sub_grade,pct_tl_nvr_dlq,loan_amnt,funded_amnt,term_num,int_rate,employlen,delinq_2yrs,total_acc,chargeoff_within_12_mths,...,x3_SD,x3_TN,x3_TX,x3_UT,x3_VA,x3_VT,x3_WA,x3_WI,x3_WV,x3_WY
0,19.0,-0.527127,1.727982,1.727986,-0.580559,1.865152,-0.304884,-0.36897,-0.515258,-0.084787,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,13.0,0.668498,2.865129,2.865135,1.722477,0.577865,-1.426903,-0.36897,0.979280,-0.084787,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,0.064992,0.590834,0.590837,-0.580559,-1.200706,1.097641,-0.36897,-0.515258,-0.084787,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,7.0,-0.276615,-1.171744,-1.171744,-0.580559,-0.423359,1.097641,-0.36897,-0.017079,-0.084787,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,24.0,0.668498,-0.725414,-0.725413,-0.580559,2.885032,-0.865894,-0.36897,0.979280,-0.084787,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1194183,15.0,-4.466997,1.727982,1.727986,1.722477,0.774793,-1.146399,-0.36897,3.636237,-0.084787,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1194184,28.0,-0.094425,0.363405,0.363407,1.722477,3.585681,1.097641,-0.36897,-0.847378,-0.084787,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1194185,27.0,0.668498,0.357719,0.357721,1.722477,3.486181,1.097641,-0.36897,-0.100109,-0.084787,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1194186,14.0,-0.584062,0.022261,0.022262,1.722477,0.552990,-1.426903,-0.36897,5.130776,-0.084787,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# rebalanced the dataset to half 0 and half 1 to prevent naiive prediction( predict all 0s to get a higher accuracy )

label1=combind_data[combind_data.label==1].sample(n=len(combind_data[combind_data.label==1]))
label0=combind_data[combind_data.label==0].sample(n=len(label1))
combind_dat=pd.concat([label1,label0],ignore_index=True)
#combind_dat.drop(combind_dat.columns[0],axis=1,inplace=True)

In [12]:
combind_dat

,sub_grade,pct_tl_nvr_dlq,loan_amnt,funded_amnt,term_num,int_rate,employlen,delinq_2yrs,total_acc,chargeoff_within_12_mths,...,x3_SD,x3_TN,x3_TX,x3_UT,x3_VA,x3_VT,x3_WA,x3_WI,x3_WV,x3_WY
0,16.0,0.668498,-0.728257,-0.728256,-0.580559,1.065002,-0.865894,-0.36897,-0.847378,-0.084787,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,26.0,-0.538514,2.160098,2.160103,1.722477,2.229987,-0.024379,0.74139,1.809579,-0.084787,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,8.0,-0.094425,-0.318883,-0.318883,-0.580559,-0.348733,0.256126,-0.36897,-0.847378,-0.084787,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13.0,0.030831,1.159408,1.159411,-0.580559,0.351916,-1.146399,-0.36897,-0.598288,-0.084787,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,14.0,-0.174133,0.590834,0.590837,-0.580559,0.339478,0.256126,-0.36897,0.148981,-0.084787,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526069,22.0,0.395212,1.162251,1.162254,1.722477,1.419473,0.536631,0.74139,1.394430,-0.084787,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
526070,3.0,-0.367711,-0.773742,-0.773742,-0.580559,-1.215217,1.097641,-0.36897,0.730191,-0.084787,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
526071,16.0,0.668498,0.636320,0.636323,-0.580559,0.874293,0.817136,-0.36897,-0.100109,-0.084787,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
526072,2.0,0.668498,-1.490145,-1.490146,-0.580559,-1.188269,1.097641,-0.36897,-1.179497,-0.084787,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
X=combind_dat.drop(columns='label')
y=combind_dat.label

In [14]:
X

,sub_grade,pct_tl_nvr_dlq,loan_amnt,funded_amnt,term_num,int_rate,employlen,delinq_2yrs,total_acc,chargeoff_within_12_mths,...,x3_SD,x3_TN,x3_TX,x3_UT,x3_VA,x3_VT,x3_WA,x3_WI,x3_WV,x3_WY
0,16.0,0.668498,-0.728257,-0.728256,-0.580559,1.065002,-0.865894,-0.36897,-0.847378,-0.084787,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,26.0,-0.538514,2.160098,2.160103,1.722477,2.229987,-0.024379,0.74139,1.809579,-0.084787,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,8.0,-0.094425,-0.318883,-0.318883,-0.580559,-0.348733,0.256126,-0.36897,-0.847378,-0.084787,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13.0,0.030831,1.159408,1.159411,-0.580559,0.351916,-1.146399,-0.36897,-0.598288,-0.084787,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,14.0,-0.174133,0.590834,0.590837,-0.580559,0.339478,0.256126,-0.36897,0.148981,-0.084787,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526069,22.0,0.395212,1.162251,1.162254,1.722477,1.419473,0.536631,0.74139,1.394430,-0.084787,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
526070,3.0,-0.367711,-0.773742,-0.773742,-0.580559,-1.215217,1.097641,-0.36897,0.730191,-0.084787,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
526071,16.0,0.668498,0.636320,0.636323,-0.580559,0.874293,0.817136,-0.36897,-0.100109,-0.084787,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
526072,2.0,0.668498,-1.490145,-1.490146,-0.580559,-1.188269,1.097641,-0.36897,-1.179497,-0.084787,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5)

In [ ]:
# the number of label 1 in train, test and valid is balanced.

In [10]:
sum(y_test)

52488

In [91]:
sum(y_valid)

52828

In [ ]:
# train model LG

In [11]:
lg=LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

In [9]:
model=LogisticRegression(solver='lbfgs')

In [8]:
lg2=LogisticRegression(C=10, class_weight=None,max_iter=1000,solver='liblinear',penalty='l1')

In [10]:
rfe = RFE(model, 20) 

In [ ]:
rfe = rfe.fit(X_train, y_train)

In [12]:
col_filter = X_train.columns[rfe.support_] #
col_filter

Index(['term_num', 'x0_Not Verified', 'x2_Individual', 'x3_CO', 'x3_DC',
       'x3_IA', 'x3_ME', 'x3_OR', 'x3_VT', 'x3_WA'],
      dtype='object')

In [12]:
names = X_train.columns
clf=RandomForestClassifier(n_estimators=10,random_state=123)# rank features importance
clf.fit(X_train, y_train) # train model
names, clf.feature_importances_
for feature in zip(names, clf.feature_importances_):
    print(feature)

('sub_grade', 0.09623515388215428)
('pct_tl_nvr_dlq', 0.060023455253253755)
('loan_amnt', 0.06829823534129643)
('funded_amnt', 0.06732433178869209)
('term_num', 0.020671566237749486)
('int_rate', 0.11128612064964838)
('employlen', 0.04737087436049878)
('delinq_2yrs', 0.01858142796305536)
('total_acc', 0.08513676645878156)
('chargeoff_within_12_mths', 0.0017515350919912159)
('delinq_amnt', 0.001292198638204131)
('annual_inc', 0.08922651645699105)
('debt_coveredratio', 0.09737900295063019)
('x0_Not Verified', 0.007497232696173383)
('x0_Source Verified', 0.0075218409088032775)
('x0_Verified', 0.008412440236384279)
('x1_car', 0.0014690002163597633)
('x1_credit_card', 0.009909815230202033)
('x1_debt_consolidation', 0.012822700850290804)
('x1_educational', 0.0)
('x1_home_improvement', 0.006495788705027606)
('x1_house', 0.0012914550518854963)
('x1_major_purchase', 0.0032179582161609957)
('x1_medical', 0.002129429022280199)
('x1_moving', 0.0013461091350718078)
('x1_other', 0.00511995252934065)

In [19]:
fea=[feature for feature in zip(names, clf.feature_importances_)]

In [22]:
sorted(fea, key=itemgetter(1), reverse=True)

[('int_rate', 0.11128612064964838),
 ('debt_coveredratio', 0.09737900295063019),
 ('sub_grade', 0.09623515388215428),
 ('annual_inc', 0.08922651645699105),
 ('total_acc', 0.08513676645878156),
 ('loan_amnt', 0.06829823534129643),
 ('funded_amnt', 0.06732433178869209),
 ('pct_tl_nvr_dlq', 0.060023455253253755),
 ('employlen', 0.04737087436049878),
 ('term_num', 0.020671566237749486),
 ('delinq_2yrs', 0.01858142796305536),
 ('x1_debt_consolidation', 0.012822700850290804),
 ('x3_CA', 0.011973772091864553),
 ('x1_credit_card', 0.009909815230202033),
 ('x3_TX', 0.009280091724931131),
 ('x0_Verified', 0.008412440236384279),
 ('x3_FL', 0.008089866776982703),
 ('x3_NY', 0.007977851604261404),
 ('x0_Source Verified', 0.0075218409088032775),
 ('x0_Not Verified', 0.007497232696173383),
 ('x1_home_improvement', 0.006495788705027606),
 ('x3_IL', 0.005925007435798067),
 ('x3_PA', 0.0057703605823057275),
 ('x3_NJ', 0.0056029429487185044),
 ('x3_OH', 0.005461873689274112),
 ('x1_other', 0.005119952529

In [14]:
lg2.fit(X_train,y_train)
trainpred=lg2.predict(X_train)
train_accuracy=sum(trainpred==y_train)/len(y_train)
train_accuracy

0.6464497978735537

In [15]:
TP_=np.logical_and(trainpred,y_train)
TN_=np.logical_and(np.logical_not(trainpred),np.logical_not(y_train))
FP_=np.logical_and(trainpred,np.logical_not(y_train))
FN_=np.logical_and(np.logical_not(trainpred),y_train)
TP=sum(TP_)
TN=sum(TN_)
FN=sum(FN_)
FP=sum(FP_)
BER = 1 - 0.5 * (TP / (TP + FN) + TN / (TN + FP))
P=TP/(TP+FP)
R=TP/(TP+FN)
F1=2*P*R/(P+R)
F10=101*P*R/(100*P+R)

In [16]:
R

0.6198540164468894

In [58]:
def default_prob(model,X_train,y_train):
    trainpred_prob=model.predict_proba(X_train)
    result=[]
    for i in range(len(y_train)):
        if trainpred_prob[i][1]>0.4:
            default_prob=1
        else:
            default_prob=0
        result.append(default_prob)
    return result

0.6368408713614071

In [22]:
result=default_prob(lg2,X_train,y_train)

In [21]:
#recall_score(result,y_train)

In [63]:
def score(result,y_train):
    accuracy=sum(result==y_train)/len(y_train)
    TP_=np.logical_and(result,y_train)
    TN_=np.logical_and(np.logical_not(result),np.logical_not(y_train))
    FP_=np.logical_and(result,np.logical_not(y_train))
    FN_=np.logical_and(np.logical_not(result),y_train)
    TP=sum(TP_)
    TN=sum(TN_)
    FN=sum(FN_)
    FP=sum(FP_)
    BER = 1 - 0.5 * (TP / (TP + FN) + TN / (TN + FP))
    P=TP/(TP+FP)
    R=TP/(TP+FN)
    F1=2*P*R/(P+R)
    F2=5*P*R/(4*P+R)
    return print("recall is:" +str(round(R,3))+"  precison is:" +str(round(P,3))+ "  accuracy is:"+ str(round(accuracy,3))+ " F1 score:"+str(round(F1,3))+ " F2 score: "+ str(round(F2,3)))

In [64]:
score(result,y_train)

recall is:0.817  precison is:0.601  accuracy is:0.637 F1 score:0.692 F2 score: 0.762


In [65]:
result_valid=default_prob(lg2,X_valid,y_valid)
score(result_valid,y_valid)

recall is:0.815  precison is:0.601  accuracy is:0.637 F1 score:0.692 F2 score: 0.761


In [66]:
result_test=default_prob(lg2,X_test,y_test)
score(result_test,y_test)

recall is:0.816  precison is:0.6  accuracy is:0.637 F1 score:0.692 F2 score: 0.761


In [6]:
clf = xgb.XGBClassifier(n_estimators=200, max_depth=13, learning_rate=0.1, subsample=0.8, silent=True, seed=1,
                           objective='binary:logistic', reg_alpha=0.5, reg_lambda=1, gamma=0, missing=None, colsample_bytree=0.3,
                       scale_pos_weight=1, min_child_weight=1, max_delta_step=0, base_socre=0.5)
clf.fit(X_train, y_train, eval_metric='auc', verbose=True, eval_set=[(X_valid, y_valid)], early_stopping_rounds=100)
clf.score(X_valid, y_valid)

[0]	validation_0-auc:0.569732
Will train until validation_0-auc hasn't improved in 100 rounds.
[1]	validation_0-auc:0.671839
[2]	validation_0-auc:0.683704
[3]	validation_0-auc:0.68099
[4]	validation_0-auc:0.67936
[5]	validation_0-auc:0.693046
[6]	validation_0-auc:0.69337
[7]	validation_0-auc:0.696697
[8]	validation_0-auc:0.700659
[9]	validation_0-auc:0.703907
[10]	validation_0-auc:0.704993
[11]	validation_0-auc:0.705146
[12]	validation_0-auc:0.704928
[13]	validation_0-auc:0.704708
[14]	validation_0-auc:0.706328
[15]	validation_0-auc:0.706647
[16]	validation_0-auc:0.706633
[17]	validation_0-auc:0.706557
[18]	validation_0-auc:0.706365
[19]	validation_0-auc:0.70642
[20]	validation_0-auc:0.706459
[21]	validation_0-auc:0.707018
[22]	validation_0-auc:0.707241
[23]	validation_0-auc:0.707164
[24]	validation_0-auc:0.707249
[25]	validation_0-auc:0.70742
[26]	validation_0-auc:0.707654
[27]	validation_0-auc:0.707716
[28]	validation_0-auc:0.707775
[29]	validation_0-auc:0.707749
[30]	validation_0-au

0.653728080596873

In [ ]:
#Run our Neural Network containing the number of inputs, 
#2 hidden layers (first: 15 nodes, second: 5 nodes) and the number of outputs which is equivalent to 2.

In [24]:
# define the keras model
model = Sequential()
model.add(Dense(15, input_dim=83, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [25]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
model.fit(X_train, y_train, epochs=50, batch_size=10)

Epoch 1/50
315644/315644 [==============================] - 83s 264us/step - loss: 0.6311 - accuracy: 0.6438
Epoch 2/50
315644/315644 [==============================] - 84s 265us/step - loss: 0.6258 - accuracy: 0.6482
Epoch 3/50
315644/315644 [==============================] - 83s 263us/step - loss: 0.6251 - accuracy: 0.6488
Epoch 4/50
315644/315644 [==============================] - 83s 262us/step - loss: 0.6247 - accuracy: 0.6494
Epoch 5/50
315644/315644 [==============================] - 82s 259us/step - loss: 0.6242 - accuracy: 0.6498
Epoch 6/50
315644/315644 [==============================] - 81s 258us/step - loss: 0.6238 - accuracy: 0.6504
Epoch 7/50
315644/315644 [==============================] - 80s 255us/step - loss: 0.6235 - accuracy: 0.6501
Epoch 8/50
315644/315644 [==============================] - 77s 243us/step - loss: 0.6232 - accuracy: 0.6514
Epoch 9/50
315644/315644 [==============================] - 76s 241us/step - loss: 0.6229 - accuracy: 0.6508
Epoch 10/50
315644/

In [16]:
#_, accuracy = model.evaluate(X_train, y_train)

716512/716512 [==============================] - 24s 33us/step


In [27]:
deeplearning_result=model.predict_classes(X_train)

In [32]:
dl_result=[]
for i in range(len(deeplearning_result)):
    dl_=deeplearning_result[i][0]
    dl_result.append(dl_)

In [34]:
TP_=np.logical_and(dl_result,y_train)
TP=sum(TP_)

In [37]:
sum(dl_result)

161793

In [35]:
TP_=np.logical_and(dl_result,y_train)
TN_=np.logical_and(np.logical_not(dl_result),np.logical_not(y_train))
FP_=np.logical_and(dl_result,np.logical_not(y_train))
FN_=np.logical_and(np.logical_not(dl_result),y_train)
TP=sum(TP_)
TN=sum(TN_)
FN=sum(FN_)
FP=sum(FP_)
BER = 1 - 0.5 * (TP / (TP + FN) + TN / (TN + FP))
P=TP/(TP+FP)
R=TP/(TP+FN)
F1=2*P*R/(P+R)
F10=101*P*R/(100*P+R)

In [36]:
R

0.6675677215550477

In [8]:
# define the keras model
model = Sequential()
model.add(Dense(15, input_dim=83, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_valid, y_valid, epochs=50, batch_size=10)

Epoch 1/50
105215/105215 [==============================] - 25s 237us/step - loss: 0.6307 - accuracy: 0.6444
Epoch 2/50
105215/105215 [==============================] - 26s 246us/step - loss: 0.6271 - accuracy: 0.6470
Epoch 3/50
105215/105215 [==============================] - 27s 253us/step - loss: 0.6261 - accuracy: 0.6484
Epoch 4/50
105215/105215 [==============================] - 26s 252us/step - loss: 0.6259 - accuracy: 0.6486
Epoch 5/50
105215/105215 [==============================] - 26s 249us/step - loss: 0.6249 - accuracy: 0.6495
Epoch 6/50
105215/105215 [==============================] - 26s 244us/step - loss: 0.6244 - accuracy: 0.6500
Epoch 7/50
105215/105215 [==============================] - 25s 242us/step - loss: 0.6239 - accuracy: 0.6504
Epoch 8/50
105215/105215 [==============================] - 26s 247us/step - loss: 0.6234 - accuracy: 0.6510
Epoch 9/50
105215/105215 [==============================] - 24s 231us/step - loss: 0.6229 - accuracy: 0.6515
Epoch 10/50
105215/

In [67]:
deeplearning_result=model.predict_classes(X_valid)
dl_result=[]
for i in range(len(deeplearning_result)):
    dl_=deeplearning_result[i][0]
    dl_result.append(dl_)
TP_=np.logical_and(dl_result,y_valid)
TP=sum(TP_)
TP_=np.logical_and(dl_result,y_valid)
TN_=np.logical_and(np.logical_not(dl_result),np.logical_not(y_valid))
FP_=np.logical_and(dl_result,np.logical_not(y_valid))
FN_=np.logical_and(np.logical_not(dl_result),y_valid)
TP=sum(TP_)
TN=sum(TN_)
FN=sum(FN_)
FP=sum(FP_)
BER = 1 - 0.5 * (TP / (TP + FN) + TN / (TN + FP))
P=TP/(TP+FP)
R=TP/(TP+FN)
F1=2*P*R/(P+R)
F10=101*P*R/(100*P+R)

In [68]:
R

0.6636467236467236

In [ ]:
logit_roc_auc = roc_auc_score(y_train, trainpred)
fpr, tpr, thresholds = roc_curve(y_train, trainpred[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()